In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# sqrtm

> Methods for computing sqrt of a matrix

In [ ]:
#| default_exp sqrtm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
from torch.autograd import Function, Variable
import numpy as np
import scipy.linalg

## Steve Li's method

In [ ]:
#| export
class MatrixSquareRoot_li(Function):
    """
    From https://github.com/steveli/pytorch-sqrtm/blob/master/sqrtm.py, which sadly does not install as a package.  LICENSE included below
    Square root of a positive definite matrix.

    NOTE: matrix square root is not differentiable for matrices with
          zero eigenvalues.
    """
    @staticmethod
    def forward(ctx, input):
        m = input.detach().cpu().numpy().astype(np.float_)  # SHH: note how this immediately switches to CPU & numpy :-( 
        sqrtm = torch.from_numpy(scipy.linalg.sqrtm(m).real).to(input)
        ctx.save_for_backward(sqrtm)
        return sqrtm

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        if ctx.needs_input_grad[0]:
            sqrtm, = ctx.saved_tensors
            sqrtm = sqrtm.data.cpu().numpy().astype(np.float_)
            gm = grad_output.data.cpu().numpy().astype(np.float_)

            # Given a positive semi-definite matrix X,
            # since X = X^{1/2}X^{1/2}, we can compute the gradient of the
            # matrix square root dX^{1/2} by solving the Sylvester equation:
            # dX = (d(X^{1/2})X^{1/2} + X^{1/2}(dX^{1/2}).
            grad_sqrtm = scipy.linalg.solve_sylvester(sqrtm, sqrtm, gm)

            grad_input = torch.from_numpy(grad_sqrtm).to(grad_output)
        return grad_input


sqrtm_li = MatrixSquareRoot_li.apply


LICENCE_LI = """
MIT License

Copyright (c) 2022 Steven Cheng-Xian Li

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

Steve Li's test code for the above:


In [ ]:
#| eval: false 
from torch.autograd import gradcheck

In [ ]:
#| eval: false 
k = torch.randn(20, 10).double()
# Create a positive definite matrix
pd_mat = (k.t().matmul(k)).requires_grad_()
sq = sqrtm_li(pd_mat)
print(sq)
test = gradcheck(sqrtm_li, (pd_mat,))
print(test)

NameError: name 'torch' is not defined

## Subhransu Maji's method(s)

From https://github.com/msubhransu/matrix-sqrt

In [ ]:
#| export

# Compute error
def compute_error(A, sA):
    normA = torch.sqrt(torch.sum(torch.sum(A * A, dim=1),dim=1))
    error = A - torch.bmm(sA, sA)
    error = torch.sqrt((error * error).sum(dim=1).sum(dim=1)) / normA
    return torch.mean(error)


def sqrt_newton_schulz_autograd(A, 
                                numIters=7, # 7 found experimentally by SHH, comparing w/ Li's method
                               ):
    """Modified from from https://people.cs.umass.edu/~smaji/projects/matrix-sqrt/
    "The drawback of the autograd approach [i.e., this approach] is that a naive implementation stores all the intermediate results. 
    Thus the memory overhead scales linearly with the number of iterations which is problematic for large matrices."
    """
    if len(A.data.shape) < 3: A = A.unsqueeze(0)
    batchSize, dim1, dim2 = A.data.shape
    assert dim1==dim2
    dim, dtype, device = dim1, A.dtype, A.device
    normA = A.mul(A).sum(dim=1).sum(dim=1).sqrt()
    Y = A.div(normA.view(batchSize, 1, 1).expand_as(A));
    I = Variable(torch.eye(dim,dim, device=device).view(1, dim, dim).
                 repeat(batchSize,1,1).type(dtype),requires_grad=False)
    Z = Variable(torch.eye(dim,dim, device=device).view(1, dim, dim).
                 repeat(batchSize,1,1).type(dtype),requires_grad=False)

    for i in range(numIters):
        T = 0.5*(3.0*I - Z.bmm(Y))
        Y = Y.bmm(T)
        Z = T.bmm(Z)
    
    sA = Y*torch.sqrt(normA).view(batchSize, 1, 1).expand_as(A)
    error = compute_error(A, sA)
    return sA, error


def sqrt_newton_schulz(A, numIters=7, calc_error=True):
    """
    Modified from https://github.com/msubhransu/matrix-sqrt/blob/cc2289a3ed7042b8dbacd53ce8a34da1f814ed2f/matrix_sqrt.py#LL72C1-L87C19
    # Forward via Newton-Schulz iterations (non autograd version)
    # Seems to be slighlty faster and has much lower memory overhead
    ...original didn't preserve device, had no batch dim checking
    """
    #batchSize = A.shape[0]
    #dim = A.shape[1]
    while len(A.data.shape) < 3: 
        A = A.unsqueeze(0)
    batchSize, dim1, dim2 = A.data.shape
    assert dim1==dim2
    dim, dtype, device = dim1, A.dtype, A.device
    normA = A.mul(A).sum(dim=1).sum(dim=1).sqrt()
    Y = A.div(normA.view(batchSize, 1, 1).expand_as(A));
    I = torch.eye(dim,dim, device=device, dtype=dtype).view(1, dim, dim).repeat(batchSize,1,1)
    Z = torch.eye(dim,dim, device=device, dtype=dtype).view(1, dim, dim).repeat(batchSize,1,1)
    for i in range(numIters):
        T = 0.5*(3.0*I - Z.bmm(Y))
        Y = Y.bmm(T)
        Z = T.bmm(Z)
        
    sA = Y*torch.sqrt(normA).view(batchSize, 1, 1).expand_as(A)
    if calc_error: 
        error = compute_error(A, sA)
        return sA, error
    else:
        return sA


def lyap_newton_schulz(z, dldz, numIters, dtype):
    """  UNUSED by FAD code
    # Backward via iterative Lyapunov solver.   
    --I don't plan use do backward passes, but including code anyway -SHH
    """
    batchSize = z.shape[0]
    dim = z.shape[1]
    normz = z.mul(z).sum(dim=1).sum(dim=1).sqrt()
    a = z.div(normz.view(batchSize, 1, 1).expand_as(z))
    I = torch.eye(dim,dim).view(1, dim, dim).repeat(batchSize,1,1).type(dtype)
    q = dldz.div(normz.view(batchSize, 1, 1).expand_as(z))
    for i in range(numIters):
        q = 0.5*(q.bmm(3.0*I - a.bmm(a)) - a.transpose(1, 2).bmm(a.transpose(1,2).bmm(q) - q.bmm(a)) )
        a = 0.5*a.bmm(3.0*I - a.bmm(a))
    dlda = 0.5*q
    return dlda    


LICENCE_SM = """
MIT License

Copyright (c) 2017 Subhransu Maji

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

## Error tests

In [ ]:
#| eval: false 
sa1, error = sqrt_newton_schulz_autograd( pd_mat.unsqueeze(0), numIters=7 ) 
sa1, error

(tensor([[[ 3.7903e+00, -1.0208e-01, -4.6511e-01,  1.7517e-01, -7.4580e-01,
           -1.4961e-01,  7.4583e-02,  2.2315e-01, -8.8865e-01, -5.6481e-01],
          [-1.0208e-01,  3.8343e+00, -9.7753e-02,  6.1448e-02, -5.9080e-01,
           -7.1447e-02,  9.9068e-01,  4.4910e-02, -7.4357e-01, -4.8293e-01],
          [-4.6511e-01, -9.7753e-02,  4.7436e+00,  2.2309e-01, -9.7453e-02,
            7.5175e-01,  6.2573e-01, -3.8497e-01, -1.1455e+00,  4.5506e-01],
          [ 1.7517e-01,  6.1448e-02,  2.2309e-01,  4.1261e+00, -8.5799e-01,
            1.0368e+00,  1.1132e-02,  7.8348e-02,  3.1722e-01,  3.4711e-01],
          [-7.4580e-01, -5.9080e-01, -9.7453e-02, -8.5799e-01,  5.0818e+00,
           -6.5439e-01,  5.0535e-01, -4.9387e-01, -5.9523e-02, -2.4595e-01],
          [-1.4961e-01, -7.1447e-02,  7.5175e-01,  1.0368e+00, -6.5439e-01,
            2.8483e+00, -1.1602e-02,  2.5504e-01, -7.5898e-01, -3.6447e-01],
          [ 7.4583e-02,  9.9068e-01,  6.2573e-01,  1.1132e-02,  5.0535e-01,
      

In [ ]:
#| eval: false 
sa2, error = sqrt_newton_schulz( pd_mat.unsqueeze(0), numIters=7 ) 
sa2, error

(tensor([[[ 3.7903e+00, -1.0208e-01, -4.6511e-01,  1.7517e-01, -7.4580e-01,
           -1.4961e-01,  7.4583e-02,  2.2315e-01, -8.8865e-01, -5.6481e-01],
          [-1.0208e-01,  3.8343e+00, -9.7753e-02,  6.1448e-02, -5.9080e-01,
           -7.1447e-02,  9.9068e-01,  4.4910e-02, -7.4357e-01, -4.8293e-01],
          [-4.6511e-01, -9.7753e-02,  4.7436e+00,  2.2309e-01, -9.7453e-02,
            7.5175e-01,  6.2573e-01, -3.8497e-01, -1.1455e+00,  4.5506e-01],
          [ 1.7517e-01,  6.1448e-02,  2.2309e-01,  4.1261e+00, -8.5799e-01,
            1.0368e+00,  1.1132e-02,  7.8348e-02,  3.1722e-01,  3.4711e-01],
          [-7.4580e-01, -5.9080e-01, -9.7453e-02, -8.5799e-01,  5.0818e+00,
           -6.5439e-01,  5.0535e-01, -4.9387e-01, -5.9523e-02, -2.4595e-01],
          [-1.4961e-01, -7.1447e-02,  7.5175e-01,  1.0368e+00, -6.5439e-01,
            2.8483e+00, -1.1602e-02,  2.5504e-01, -7.5898e-01, -3.6447e-01],
          [ 7.4583e-02,  9.9068e-01,  6.2573e-01,  1.1132e-02,  5.0535e-01,
      

In [ ]:
#| eval: false 
sa1 - sq

tensor([[[-4.4797e-05, -3.1908e-05, -9.0556e-07,  4.1434e-05, -2.1449e-05,
          -8.8734e-05, -1.8313e-06,  1.5652e-05, -4.6569e-05, -2.6264e-05],
         [-3.1908e-05, -2.2827e-05, -7.7784e-07,  2.9361e-05, -1.5343e-05,
          -6.2912e-05, -1.1153e-06,  1.0995e-05, -3.3114e-05, -1.8642e-05],
         [-9.0556e-07, -7.7784e-07, -1.9497e-07,  6.3666e-07, -5.1983e-07,
          -1.4083e-06,  2.1449e-07,  1.1260e-07, -8.6786e-07, -4.4392e-07],
         [ 4.1434e-05,  2.9361e-05,  6.3666e-07, -3.8552e-05,  1.9741e-05,
           8.2510e-05,  1.9799e-06, -1.4708e-05,  4.3156e-05,  2.4391e-05],
         [-2.1449e-05, -1.5343e-05, -5.1983e-07,  1.9741e-05, -1.0312e-05,
          -4.2298e-05, -7.5408e-07,  7.3947e-06, -2.2262e-05, -1.2533e-05],
         [-8.8734e-05, -6.2912e-05, -1.4083e-06,  8.2510e-05, -4.2298e-05,
          -1.7660e-04, -4.1762e-06,  3.1447e-05, -9.2404e-05, -5.2213e-05],
         [-1.8313e-06, -1.1153e-06,  2.1449e-07,  1.9799e-06, -7.5408e-07,
          -4.1762e-

In [ ]:
#| eval: false 
sa2 - sq

tensor([[[-4.4797e-05, -3.1908e-05, -9.0556e-07,  4.1434e-05, -2.1449e-05,
          -8.8734e-05, -1.8313e-06,  1.5652e-05, -4.6569e-05, -2.6264e-05],
         [-3.1908e-05, -2.2827e-05, -7.7784e-07,  2.9361e-05, -1.5343e-05,
          -6.2912e-05, -1.1153e-06,  1.0995e-05, -3.3114e-05, -1.8642e-05],
         [-9.0556e-07, -7.7784e-07, -1.9497e-07,  6.3666e-07, -5.1983e-07,
          -1.4083e-06,  2.1449e-07,  1.1260e-07, -8.6786e-07, -4.4392e-07],
         [ 4.1434e-05,  2.9361e-05,  6.3666e-07, -3.8552e-05,  1.9741e-05,
           8.2510e-05,  1.9799e-06, -1.4708e-05,  4.3156e-05,  2.4391e-05],
         [-2.1449e-05, -1.5343e-05, -5.1983e-07,  1.9741e-05, -1.0312e-05,
          -4.2298e-05, -7.5408e-07,  7.3947e-06, -2.2262e-05, -1.2533e-05],
         [-8.8734e-05, -6.2912e-05, -1.4083e-06,  8.2510e-05, -4.2298e-05,
          -1.7660e-04, -4.1762e-06,  3.1447e-05, -9.2404e-05, -5.2213e-05],
         [-1.8313e-06, -1.1153e-06,  2.1449e-07,  1.9799e-06, -7.5408e-07,
          -4.1762e-

## Speed & device tests

In [ ]:
#| eval: false 
from aeiou.core import get_device

In [ ]:
#| eval: false 
device = get_device()
print('device = ',device)
n,m = 1000, 1000
with torch.no_grad(): 
    k = torch.randn(n, m, device=device)
    pd_mat2 = (k.t().matmul(k)) # Create a positive definite matrix, no grad

device =  mps


In [ ]:
#| eval: false 
#%%timeit
sq2 = sqrtm_li(pd_mat2)

1.7 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Result of `%%timeit`:

`1.7 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)`

In [ ]:
#| eval: false 
#%%timeit
sq3 = sqrt_newton_schulz(pd_mat2.unsqueeze(0))

Result of `%%timeit`:

`13 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)`

## Thus, we'll put a wrapper around our method of choice:

In [ ]:
#|export 
def sqrtm(A, numIters=7):
    "wrapper function for matrix sqrt algorithm of choice"
    return sqrt_newton_schulz(A, numIters=numIters, calc_error=False)

In [ ]:
#| eval: false 
sqrtm(pd_mat2) 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()